In [1]:
import cv2
import threading
import numpy as np
from metavision_core.event_io import EventsIterator
from metavision_sdk_core import PeriodicFrameGenerationAlgorithm
from metavision_sdk_ui import EventLoop, BaseWindow, Window, UIAction, UIKeyEvent
from pylablib.devices import uc480  # uc480 DLLs needed for Thorlabs camera
import matplotlib.pyplot as plt
import argparse
import h5py
import time

In [2]:
import os
import threading
import time

class ThorlabsCam:
    def __init__(self):
        notebook_dir = os.getcwd()
        self.save_folder = os.path.join(notebook_dir, "frames")
        if not os.path.exists(self.save_folder):
            os.makedirs(self.save_folder)

        self.cam = uc480.UC480Camera()
        self.is_acquiring = False
        self.frame_count = 0
        self.save_thread = None
        
    def start_saving(self):
        self.save_thread = threading.Thread(target=self.save_frames)
        self.save_thread.start()
        print("Thorlabs frame saving started")

    def start_acquisition(self):
        self.is_acquiring = True
        self.cam.start_acquisition()
        print("Thorlabs acquisition started")

    def stop_acquisition(self):
        self.is_acquiring = False
        self.cam.stop_acquisition()
        self.cam.close()
        print("Thorlabs acquisition stopped")

    def save_frames(self):
        self.frames_list=[]
        while self.is_acquiring:
            frame = self.cam.snap()
            if frame is not None:
                self.frames_list.append(frame)
                #filename = os.path.join(self.save_folder, f"frame_{self.frame_count}.png")
                #cv2.imwrite(filename, frame)
                self.frame_count += 1
                #time.sleep(1 / 15)  # Delay to achieve 15 FPS
            else:
                print("Failed to capture frame")

    def stop_saving(self):
        self.is_acquiring = False
        if self.save_thread:
            self.save_thread.join()
        print("Thorlabs frame saving stopped")

In [3]:
class thread_cam:
    def __init__(self):
        self.thorlabs_cam = ThorlabsCam()
        self.thorlabs_thread = None

    def start(self):
        self.thorlabs_thread = threading.Thread(target=self.thorlabs_cam.start_acquisition)
        self.thorlabs_thread.start()
        self.thorlabs_cam.start_saving()
        self.start_time = time.time()  # Start saving frames from Thorlabs camera

    def stop(self):   
        self.thorlabs_cam.stop_acquisition()
        self.end_time = time.time()
        self.thorlabs_cam.stop_saving()

    def wait_for_timeout(self, timeout):
        time.sleep(timeout)
        self.stop()

# Example usage:
if __name__ == "__main__":
    multicam = thread_cam()
    multicam.start()
    multicam.wait_for_timeout(20) 
    print('time used to capture all the frames', multicam.end_time - multicam.start_time)
    print('number of frames in that time', multicam.thorlabs_cam.frame_count)


Thorlabs frame saving started


Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\INESC\anaconda3\envs\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\INESC\anaconda3\envs\python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\INESC\AppData\Local\Temp\ipykernel_1700\3577733818.py", line 24, in start_acquisition
  File "C:\Users\INESC\anaconda3\envs\python38\lib\site-packages\pylablib\devices\uc480\uc480.py", line 401, in start_acquisition
    self.lib.is_CaptureVideo(self.hcam,uc480_defs.LIVEFREEZE.IS_DONT_WAIT,check=True)
  File "C:\Users\INESC\anaconda3\envs\python38\lib\site-packages\pylablib\devices\uc480\uc480_lib.py", line 434, in wrapped_func
    res_errcheck(result,func,args)
  File "C:\Users\INESC\anaconda3\envs\python38\lib\site-packages\pylablib\devices\uc480\uc480_lib.py", line 35, in errchecker
    raise uc480LibError(func.__name__,result)
pylablib.devices.uc480.uc480_lib.uc480Li

Thorlabs acquisition stopped
Thorlabs frame saving stopped
time used to capture all the frames 20.133084774017334
number of frames in that time 214


In [4]:
101/10

10.1

### live video attempt

In [5]:
# init camera
cam = uc480.UC480Camera()

# params
cam.start_live_video(framerate = "10Hz")

while cam.is_open:
     
     frame = cam.grab_image(timeout='100s', copy=True, exposure_time='10ms')
     
     frame1 = np.stack((frame,) * 3,-1) #make frame as 1 channel image
     frame1 = frame1.astype(np.uint8)

     gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

     #now u can apply opencv features

     cv2.imshow('Camera', gray)
     
     if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cam.close()
cv2.destroyAllWindows()

AttributeError: 'UC480Camera' object has no attribute 'start_live_video'

Não consigo abrir live video 